# Homework: Sonar signal classification
In this workshop you will train a binary classification neural network to differentiate if sonar signals bounced against a metal cylinder or a rock.

[Info of the data](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks))

Goal: Get an accuracy higher than 0.80 in the validation set

## Get the data from Google Drive

In [1]:
# Import libraries to interact with Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
# Authenticate with your Google account to get access to the data
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# Download data
download = drive.CreateFile({'id': '1rw5l3jCo2vlNc8NLrMk3KPZr6HsjNRCY'})
download.GetContentFile('sonar.csv')

In [4]:
ls

sample_data/  sonar.csv


## Do some magic below! ;)

Paso 1: Cargar los datos
Primero, cargaremos el conjunto de datos desde el archivo sonar.csv

In [5]:
# Importar las bibliotecas necesarias
import pandas as pd

# Cargar los datos
datos = pd.read_csv('sonar.csv', header=None)
datos.head()


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


Paso 2: Explorar y preprocesar los datos

In [6]:
# Verificar valores faltantes
print(datos.isnull().sum().sum())

# Mostrar información básica sobre el conjunto de datos
print(datos.info())

# Mostrar estadísticas básicas
print(datos.describe())

# Mostrar las primeras filas del conjunto de datos
datos.head()


0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


separaremos las características y las etiquetas, y codificaremos las etiquetas.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Separar características y etiquetas
X = datos.iloc[:, :-1].values
y = datos.iloc[:, -1].values

# Codificar etiquetas (R para roca, M para metal)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir el conjunto de datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


Paso 3: Construir y compilar el modelo de red neuronal

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Definir el modelo
modelo = Sequential()
modelo.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(32, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
modelo.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


Entrenaremos el modelo usando los datos de entrenamiento y lo validaremos usando los datos de validación

In [9]:
# Entrenar el modelo
historia = modelo.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_val, y_val), verbose=1)


Epoch 1/100
17/17 [==============================] - 1s 15ms/step - loss: 0.7205 - accuracy: 0.4940 - val_loss: 0.6622 - val_accuracy: 0.7381
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6990 - accuracy: 0.5060 - val_loss: 0.6539 - val_accuracy: 0.6667
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6928 - accuracy: 0.5602 - val_loss: 0.6498 - val_accuracy: 0.7619
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7157 - accuracy: 0.4699 - val_loss: 0.6552 - val_accuracy: 0.7143
Epoch 5/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6684 - accuracy: 0.5843 - val_loss: 0.6572 - val_accuracy: 0.7619
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6828 - accuracy: 0.5602 - val_loss: 0.6481 - val_accuracy: 0.6905
Epoch 7/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6890 - accuracy: 0.5783 - val_loss: 0.6342 - val_accuracy: 0.6429
Epoch 8/100


Evaluaremos el modelo para ver si cumple con el requisito de precisión

In [10]:
# Evaluar el modelo en el conjunto de validación
perdida_val, precision_val = modelo.evaluate(X_val, y_val, verbose=0)
print(f'Precisión de validación: {precision_val:.2f}')


Precisión de validación: 0.86


En los primeras epoch, la precisión del modelo es baja (~50%) y la pérdida es alta (~0.72).

Con el tiempo, tanto la precisión del entrenamiento como la de la validación mejoran, mientras que la pérdida disminuye.

A medida que el entrenamiento progresa, las métricas de precisión y pérdida se estabilizan.

Se observa que la precisión de validación supera el 80% alrededor de la epoch 12, y sigue mejorando, alcanzando un máximo de alrededor del 90% en varias ocasiones.
